# Dectect Eilands on a map layer

This notebook takes all the tiles of a layer map and uses the Azure Custom Vision model downloaded from the Azure portal to detect the Eilands on the map.

Download the model from the Azure portal: https://www.customvision.ai/projects/2d90f277-ac1c-490c-9b14-34fee8011de7#/performance
Models must be trained with a compact setting. 

In [1]:
# Import ALL dependencies
import os
import geopandas as gpd
from shapely.geometry import box
import pandas as pd
import tensorflow as tf
import os
from PIL import Image
import numpy as np
import object_detection as od
import os
import tensorflow as tf
import cv2 
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
from ObjectDetection import ObjectDetection


## Calculate tile areas

In [2]:
# This is the only line you need to change to run on your own images
layer_name = "ign_belgium"

In [3]:
# Compile the tiles in a dataframe

layer_path = "map_layers/{}".format(layer_name)
layer_tiles_path = layer_path + "/images/tiles/"
zoom_levels = os.listdir(layer_tiles_path)
tiles = []

for zoom_level in zoom_levels:
    print("Processing zoom level {}".format(zoom_level))
    y_folders = os.listdir(layer_tiles_path + zoom_level)
    for y_folder in y_folders:
        x_files = os.listdir(layer_tiles_path + zoom_level + "/" + y_folder)
        for x_file in x_files:
            if x_file.endswith(".jpg"):
                tile_path = layer_tiles_path + zoom_level + "/" + y_folder + "/" + x_file
                tiles.append([int(zoom_level), int(y_folder), int(x_file.replace(".jpg","")), tile_path])
            
# make a dataframe
tiles = pd.DataFrame(tiles, columns=["z", "y", "x", "path"])

print("There are {} tiles in the map layer".format(len(tiles)))

Processing zoom level 15
There are 21837 tiles in the map layer


## Clean Tiles

In [41]:
# Go through each tile, if the tile is mostly white then delete it 
# (this is to remove empty tiles)
def is_valid_tile(tile_path):
    img = cv2.imread(tile_path)
    np_img = np.array(img)
    # if all the pixels are white
    if np.all(np_img == [255, 255, 255]):
        return False
    else:
        return True
    
tiles["valid"] = tiles["path"].apply(lambda x: is_valid_tile(x))


        

In [ ]:
# print the number of valid tiles 
print("There are {} valid tiles in the map layer".format(len(tiles[tiles["valid"] != False])))
# delete the invalid tiles
to_delete = tiles[tiles["valid"] == False]["path"].tolist()
for tile in to_delete:
    print("Deleting {}".format(tile))
    os.remove(tile)

## Export tiles to dataframe

In [4]:
import tile_coordinates as tc
import json

# only keep z = 15
tiles = tiles[tiles["z"] == 15]

tiles["bbox"] = tiles.apply(lambda row: tc.tile_bbox(
    row["z"], row["y"], row["x"]), axis=1)
tiles["west"] = tiles["bbox"].apply(lambda bbox: bbox.west)
tiles["east"] = tiles["bbox"].apply(lambda bbox: bbox.east)
tiles["north"] = tiles["bbox"].apply(lambda bbox: bbox.north)
tiles["south"] = tiles["bbox"].apply(lambda bbox: bbox.south)
tiles["geometry"] = tiles.apply(lambda row: box(
    row["west"], row["south"], row["east"], row["north"]), axis=1)
tile_frame_gpd = gpd.GeoDataFrame(tiles, geometry="geometry")
# drop the bbox column
tile_frame_gpd = tile_frame_gpd.drop(columns=["bbox"])
# drop geometries that are not Bounding Boxes
tile_frame_gpd = tile_frame_gpd[tile_frame_gpd["geometry"].geom_type == "Polygon"]
# exoty to escel
tile_frame_gpd.to_file(
    layer_path + "/predictions/tile_grid.shp", driver='ESRI Shapefile')

# Detect Images

In [5]:
graph_def = tf.compat.v1.GraphDef()
labels = []

# These are set to the default names from exported models, update as needed.
filename = layer_path+"\models\\azure\model.pb"
labels_filename = layer_path+"\models\\azure\labels.txt"

# Import the TF graph
with tf.io.gfile.GFile(filename, 'rb') as f:
    graph_def.ParseFromString(f.read())
    tf.import_graph_def(graph_def, name='')

# Create a list of labels.
with open(labels_filename, 'rt') as lf:
    for l in lf:
        labels.append(l.strip())
        
class TFObjectDetection(ObjectDetection):
    """Object Detection class for TensorFlow"""

    def __init__(self, graph_def, labels):
        super(TFObjectDetection, self).__init__(labels)
        self.graph = tf.compat.v1.Graph()
        with self.graph.as_default():
            input_data = tf.compat.v1.placeholder(tf.float32, [1, None, None, 3], name='Placeholder')
            tf.import_graph_def(graph_def, input_map={"Placeholder:0": input_data}, name="")

    def predict(self, preprocessed_image):
        inputs = np.array(preprocessed_image, dtype=float)[:, :, (2, 1, 0)]  # RGB -> BGR

        with tf.compat.v1.Session(graph=self.graph) as sess:
            output_tensor = sess.graph.get_tensor_by_name('model_outputs:0')
            outputs = sess.run(output_tensor, {'Placeholder:0': inputs[np.newaxis, ...]})
            return outputs[0]
        
od_model = TFObjectDetection(graph_def, labels)


In [6]:
import tile_coordinates as tc
import json 

# only keep z = 15
tiles = tiles[tiles["z"] == 15]
detected_eilands = gpd.GeoDataFrame(columns=["geometry", "prob", "tile"], crs="EPSG:4326", geometry="geometry")

# if the predictions folder does not exist, create it
if not os.path.exists(layer_path + "/predictions"):
    os.makedirs(layer_path + "/predictions")

for index, tile in tiles.iterrows(): 
    image = Image.open(tile["path"])
    print("Processing tile {}".format(tile["path"]), index)
    # clear the plt 
    #plt.clf()
    # add the image to the plot
    #plt.imshow(image)
    # Run the model
    predictions = od_model.predict_image(image)    
    # Calculate the bounding boox
    bbox = tc.tile_bbox(tile["z"], tile["y"], tile["x"])
    bbox.south = bbox.south
    bbox.north = bbox.north
    
    # For the popp maps 
    #bbox.south = -bbox.south
    #bbox.north = -bbox.north
        
    # tile width / bbox width
    y_resize_factor = image.width / (bbox.east - bbox.west)
    x_resize_factor = image.height / (bbox.north - bbox.south)
    # Draw bounding boxes
    for prediction in predictions:
        
        
        if prediction['probability'] > 0:
            bounding_box = prediction['boundingBox']
            left = bounding_box['left'] * image.width
            top = bounding_box['top'] * image.height
            width = bounding_box['width'] * image.width
            height = bounding_box['height'] * image.height
            bottom = top + height
            
            gpd_left = left / y_resize_factor + bbox.west
            gpd_top = bbox.north - top / x_resize_factor
            # for the popp karte:
            #gpd_top = bbox.south + top/ x_resize_factor
            gpd_width = width / y_resize_factor
            
            rect = Rectangle((left, top), width, height, linewidth=1, edgecolor='r', facecolor='none')
            gpd_polygon = box(gpd_left, gpd_top, gpd_left + gpd_width, gpd_top - (height / x_resize_factor) )
            # For popp map:
            #gpd_polygon = box(gpd_left, gpd_top, gpd_left + gpd_width, gpd_top + (height / x_resize_factor))
            detected_eilands = pd.concat([detected_eilands, gpd.GeoDataFrame({"geometry": [gpd_polygon], 
                                                                            "prob": [prediction['probability']],
                                                                            "tile": [tile["path"]],
                                                                            }, crs="EPSG:4326")])
            # add the rectangle to the plot
            

            # clear the plot 
            #plt.gca().add_patch(rect)
            #show image
            #plt.show()
            # Add label
            #plt.text(left, top, prediction['tagName'], fontsize=8, color='r', verticalalignment='bottom')
    
    # export the image            
    #plt.savefig(r"C:\Users\jaddh\code_projects\EILearn\map_layers\{}\images\predicted\{}.png".format(layer_name, str(tile["y"])+"_"+str(tile["x"])))

            
    # save the predictions json of the image 
    predictions = json.dumps(predictions)
    with open(tile["path"].replace(".jpg", ".json"), "w") as f:
        f.write(predictions)

    # every 10 tiles, save the predictions, or if it is the last tile
    if index % 20 == 0 or index == len(tiles) - 1:
        print("total number of detected eilands: {}".format(len(detected_eilands)))
        # export the geodataframe
        detected_eilands.to_file(layer_path + "/predictions/detected_eilands.shp")

Processing tile map_layers/ign_belgium/images/tiles/15/16615/10945.jpg 0
total number of detected eilands: 0
Processing tile map_layers/ign_belgium/images/tiles/15/16615/10946.jpg 1


c:\Users\jaddh\code_projects\EILearn\venv\Lib\site-packages\geopandas\io\file.py:608: UserWarning: You are attempting to write an empty DataFrame to file. For some drivers, this operation may fail.
  _to_file_fiona(df, filename, driver, schema, crs, mode, **kwargs)


Processing tile map_layers/ign_belgium/images/tiles/15/16615/10947.jpg 2
Processing tile map_layers/ign_belgium/images/tiles/15/16615/10948.jpg 3
Processing tile map_layers/ign_belgium/images/tiles/15/16615/10949.jpg 4
Processing tile map_layers/ign_belgium/images/tiles/15/16615/10950.jpg 5
Processing tile map_layers/ign_belgium/images/tiles/15/16615/10951.jpg 6
Processing tile map_layers/ign_belgium/images/tiles/15/16615/10952.jpg 7
Processing tile map_layers/ign_belgium/images/tiles/15/16615/10953.jpg 8
Processing tile map_layers/ign_belgium/images/tiles/15/16615/10954.jpg 9
Processing tile map_layers/ign_belgium/images/tiles/15/16615/10955.jpg 10
Processing tile map_layers/ign_belgium/images/tiles/15/16615/10956.jpg 11


C:\Users\jaddh\AppData\Local\Temp\ipykernel_14684\300741896.py:55: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  detected_eilands = pd.concat([detected_eilands, gpd.GeoDataFrame({"geometry": [gpd_polygon],


Processing tile map_layers/ign_belgium/images/tiles/15/16615/10957.jpg 12
Processing tile map_layers/ign_belgium/images/tiles/15/16615/10958.jpg 13
Processing tile map_layers/ign_belgium/images/tiles/15/16615/10959.jpg 14
Processing tile map_layers/ign_belgium/images/tiles/15/16615/10960.jpg 15
Processing tile map_layers/ign_belgium/images/tiles/15/16615/10961.jpg 16
Processing tile map_layers/ign_belgium/images/tiles/15/16615/10962.jpg 17
Processing tile map_layers/ign_belgium/images/tiles/15/16615/10963.jpg 18
Processing tile map_layers/ign_belgium/images/tiles/15/16615/10964.jpg 19
Processing tile map_layers/ign_belgium/images/tiles/15/16615/10965.jpg 20
total number of detected eilands: 4
Processing tile map_layers/ign_belgium/images/tiles/15/16615/10966.jpg 21
Processing tile map_layers/ign_belgium/images/tiles/15/16615/10967.jpg 22
Processing tile map_layers/ign_belgium/images/tiles/15/16615/10968.jpg 23
Processing tile map_layers/ign_belgium/images/tiles/15/16615/10969.jpg 24
Pr